<img src="https://github.com/AmsterdamUMC/AmsterdamUMCdb/blob/master/img/logos_combined_lccci_amds.png?raw=1" alt="Logos Laboratory for Critical Care Computational Intelligence / Amsterdam Medical Data Science" width=256px>

# AmsterdamUMCdb - Freely Accessible ICU Database

version 1.0.2 March 2020  
Copyright &copy; 2003-2022 Amsterdam UMC - Amsterdam Medical Data Science

## Reason for admission
The reason for admission is the primary diagnosis leading to ICU admission. In AmsterdamUMCdb different classification systems were used to document the reason for admission.

### Reason for admission diagnosis systems
- an internally used diagnosis list, that shows similarity with the APACHE II admission diagnosis ( 'legacy' diagnosis system)
- APACHE II admission (primary) diagnosis
- APACHE IV admission diagnosis
- NICE, the Foundation Dutch Intensive Care Evaluation (Stichting NICE), a national registry which uses APACHE II and IV for primary and secondary diagnosis

In addition, reason for admission has been documented as a free text description for many patients as well, but this is currently not available in the freely accessible version due to re-identification risks.

### Use cases
- Differentiating between surgical and medical admissions
- Selecting specific types of patients (e.g. trauma, cardiac arrest)

### Specific categories
#### Sepsis
Sepsis is a common reason for ICU admission. However, even though all reasons for admission system have a sepsis category, it's often not documented on that category, but instead other conditions were chosen (e.g. sepsis due to intestinal perforation will have the abdominal surgery category selected). To ameliorate this situation, the code in this notebook code uses the following definitions of sepsis:
- sepsis_at_admission flagged in the admission form by the attending physician
- the admission diagnosis, medical or surgical, is considered a severe infection, e.g., gastro-intestinal perforation, cholangitis, meningitis 
- use of antibiotics that are not used prophylactically after surgery 
- use of antibiotics and cultures drawn within 6 hours of admission 

**To do**: sepsis based on Sepsis III-criteria (e.g. qSOFA)

## Imports

In [1]:
%matplotlib inline
import psycopg2
import pandas as pd
import numpy as np
import re

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib as mpl

import io
from IPython.display import display, HTML, Markdown

## Display settings

In [2]:
#matplotlib settings for image size
#needs to be in a different cell from %matplotlib inline
plt.style.use('seaborn-darkgrid')
plt.rcParams["figure.dpi"] = 288
plt.rcParams["figure.figsize"] = [16, 12]
plt.rcParams["font.size"] = 12

pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.options.display.max_colwidth = 1000

## Connection settings

In [3]:
#Modify config.ini in the root folder of the repository to change the settings to connect to your postgreSQL database
import configparser
import os
config = configparser.ConfigParser()

if os.path.isfile('../../config.ini'):
    config.read('../../config.ini')
else:
    config.read('../../config.SAMPLE.ini')

#Open a connection to the postgres database:
con = psycopg2.connect(database=config['psycopg2']['database'], 
                       user=config['psycopg2']['username'], password=config['psycopg2']['password'], 
                       host=config['psycopg2']['host'], port=config['psycopg2']['port'])
con.set_client_encoding('WIN1252') #Uses code page for Dutch accented characters.
con.set_session(autocommit=True)

cursor = con.cursor()
cursor.execute('SET SCHEMA \'amsterdamumcdb\''); #set search_path to amsterdamumcdb schema

In [4]:
#loads the admissions table for stratification on admission unit 
sql = """
SELECT * FROM admissions;
"""
admissions = pd.read_sql(sql,con)

In [5]:
#No. patients
no_patients_total = admissions.patientid.nunique()
no_patients_ICU = admissions[admissions['location'].str.contains('IC')].patientid.nunique()
no_patients_MCU = admissions[admissions['location'].str.contains('^MC$')].patientid.nunique()

#No. admissions
no_admissions_total = admissions.admissionid.nunique()
no_admissions_ICU = admissions[admissions['location'].str.contains('IC')].admissionid.nunique()
no_admissions_MCU = admissions[admissions['location'].str.contains('^MC$')].admissionid.nunique()

## Admission types
- Surgical
- Medical
- Urgent
- Elective

## Admissions with reasons for admission
Creates a combined_diagnoses dataframe containing the admissions table joined with the diagnoses from the listitems table

In [6]:
with open('../../amsterdamumcdb/sql/diagnosis/reason_for_admission.sql') as sql_file:
    sql = sql_file.read()
    display(Markdown("```sql\n" + sql + "\n```"))
combined_diagnoses = pd.read_sql(sql,con)
combined_diagnoses.head()

```sql
WITH diagnosis_groups AS (
    SELECT admissionid,
        item,
        CASE
            WHEN itemid IN (
                18669, --NICE APACHEII diagnosen
                18671 --NICE APACHEIV diagnosen
            )
            THEN split_part(value, ' - ', 1)
            -- 'e.g. 'Non-operative cardiovascular - Anaphylaxis' -> Non-operative cardiovascular
            ELSE value
        END as diagnosis_group,
        valueid as diagnosis_group_id,
        ROW_NUMBER() OVER(PARTITION BY admissionid
        ORDER BY
            CASE --prefer NICE > APACHE IV > II > D
                WHEN itemid = 18671 THEN 6 --NICE APACHEIV diagnosen
                WHEN itemid = 18669 THEN 5 --NICE APACHEII diagnosen
                WHEN itemid BETWEEN 16998 AND 17017 THEN 4 --APACHE IV diagnosis
                WHEN itemid BETWEEN 18589 AND 18602 THEN 3 --APACHE II diagnosis
                WHEN itemid BETWEEN 13116 AND 13145 THEN 2 --D diagnosis ICU
                WHEN itemid BETWEEN 16642 AND 16673 THEN 1 --DMC diagnosis Medium Care
            END DESC,
            updatedat DESC) AS rownum --prioritizes diagnosis with last update time
    FROM listitems
    WHERE itemid IN (
        --MAIN GROUP - LEVEL 0
        13110, --D_Hoofdgroep
        16651, --DMC_Hoofdgroep, Medium Care

        18588, --Apache II Hoofdgroep
        16997, --APACHE IV Groepen

        18669, --NICE APACHEII diagnosen
        18671 --NICE APACHEIV diagnosen
    )
),diagnosis_subgroups AS (
    SELECT admissionid,
        item,
        value as diagnosis_subgroup,
        valueid as diagnosis_subgroup_id,
        ROW_NUMBER() OVER(PARTITION BY admissionid
        ORDER BY
            updatedat DESC) AS rownum --prioritizes diagnosis with last update time
    FROM listitems
    WHERE itemid IN (
        --SUB GROUP - LEVEL 1
        13111, --D_Subgroep_Thoraxchirurgie
        16669, --DMC_Subgroep_Thoraxchirurgie
        13112, --D_Subgroep_Algemene chirurgie
        16665, --DMC_Subgroep_Algemene chirurgie
        13113, --D_Subgroep_Neurochirurgie
        16667, --DMC_Subgroep_Neurochirurgie
        13114, --D_Subgroep_Neurologie
        16668, --DMC_Subgroep_Neurologie
        13115, --D_Subgroep_Interne geneeskunde
        16666 --DMC_Subgroep_Interne geneeskunde
    )
), diagnoses AS (
    SELECT admissionid,
        item,
        CASE
            WHEN itemid IN (
                18669, --NICE APACHEII diagnosen
                18671 --NICE APACHEIV diagnosen
            )
            THEN split_part(value, ' - ', 2)
            -- 'e.g. 'Non-operative cardiovascular - Anaphylaxis' -> Anaphylaxis
            ELSE value
        END as diagnosis,
        CASE
            WHEN itemid IN (
                --SURGICAL
                13116, --D_Thoraxchirurgie_CABG en Klepchirurgie
                16671, --DMC_Thoraxchirurgie_CABG en Klepchirurgie
                13117, --D_Thoraxchirurgie_Cardio anders
                16672, --DMC_Thoraxchirurgie_Cardio anders
                13118, --D_Thoraxchirurgie_Aorta chirurgie
                16670, --DMC_Thoraxchirurgie_Aorta chirurgie
                13119, --D_Thoraxchirurgie_Pulmonale chirurgie
                16673, --DMC_Thoraxchirurgie_Pulmonale chirurgie

                --Not surgical: 13141, --D_Algemene chirurgie_Algemeen
                --Not surgical: 16642, --DMC_Algemene chirurgie_Algemeen
                13121, --D_Algemene chirurgie_Buikchirurgie
                16643, --DMC_Algemene chirurgie_Buikchirurgie
                13123, --D_Algemene chirurgie_Endocrinologische chirurgie
                16644, --DMC_Algemene chirurgie_Endocrinologische chirurgie
                13145, --D_Algemene chirurgie_KNO/Overige
                16645, --DMC_Algemene chirurgie_KNO/Overige
                13125, --D_Algemene chirurgie_Orthopedische chirurgie
                16646, --DMC_Algemene chirurgie_Orthopedische chirurgie
                13122, --D_Algemene chirurgie_Transplantatie chirurgie
                16647, --DMC_Algemene chirurgie_Transplantatie chirurgie
                13124, --D_Algemene chirurgie_Trauma
                16648, --DMC_Algemene chirurgie_Trauma
                13126, --D_Algemene chirurgie_Urogenitaal
                16649, --DMC_Algemene chirurgie_Urogenitaal
                13120, --D_Algemene chirurgie_Vaatchirurgie
                16650, --DMC_Algemene chirurgie_Vaatchirurgie

                13128, --D_Neurochirurgie _Vasculair chirurgisch
                16661, --DMC_Neurochirurgie _Vasculair chirurgisch
                13129, --D_Neurochirurgie _Tumor chirurgie
                16660, --DMC_Neurochirurgie _Tumor chirurgie
                13130, --D_Neurochirurgie_Overige
                16662, --DMC_Neurochirurgie_Overige

                18596, --Apache II Operatief  Gastr-intenstinaal
                18597, --Apache II Operatief Cardiovasculair
                18598, --Apache II Operatief Hematologisch
                18599, --Apache II Operatief Metabolisme
                18600, --Apache II Operatief Neurologisch
                18601, --Apache II Operatief Renaal
                18602, --Apache II Operatief Respiratoir

                17008, --APACHEIV Post-operative cardiovascular
                17009, --APACHEIV Post-operative gastro-intestinal
                17010, --APACHEIV Post-operative genitourinary
                17011, --APACHEIV Post-operative hematology
                17012, --APACHEIV Post-operative metabolic
                17013, --APACHEIV Post-operative musculoskeletal /skin
                17014, --APACHEIV Post-operative neurologic
                17015, --APACHEIV Post-operative respiratory
                17016, --APACHEIV Post-operative transplant
                17017 --APACHEIV Post-operative trauma

            ) THEN 1
            WHEN itemid = 18669 AND valueid BETWEEN 1 AND 26 THEN 1 --NICE APACHEII diagnosen
            WHEN itemid = 18671 AND valueid BETWEEN 222 AND 452 THEN 1 --NICE APACHEIV diagnosen
            ELSE 0
        END AS surgical,
        valueid as diagnosis_id,
        CASE
                WHEN itemid = 18671 THEN 'NICE APACHE IV'
                WHEN itemid = 18669 THEN 'NICE APACHE II'
                WHEN itemid BETWEEN 16998 AND 17017 THEN 'APACHE IV'
                WHEN itemid BETWEEN 18589 AND 18602 THEN 'APACHE II'
                WHEN itemid BETWEEN 13116 AND 13145 THEN 'Legacy ICU'
                WHEN itemid BETWEEN 16642 AND 16673 THEN 'Legacy MCU'
        END AS diagnosis_type,
        ROW_NUMBER() OVER(PARTITION BY admissionid
        ORDER BY
            CASE --prefer NICE > APACHE IV > II > D
                WHEN itemid = 18671 THEN 6 --NICE APACHEIV diagnosen
                WHEN itemid = 18669 THEN 5 --NICE APACHEII diagnosen
                WHEN itemid BETWEEN 16998 AND 17017 THEN 4 --APACHE IV diagnosis
                WHEN itemid BETWEEN 18589 AND 18602 THEN 3 --APACHE II diagnosis
                WHEN itemid BETWEEN 13116 AND 13145 THEN 2 --D diagnosis ICU
                WHEN itemid BETWEEN 16642 AND 16673 THEN 1 --DMC diagnosis Medium Care
            END DESC,
            updatedat DESC) AS rownum --prioritizes diagnosis with last update time
    FROM listitems
    WHERE itemid IN (
        -- Diagnosis - LEVEL 2
        --SURGICAL
        13116, --D_Thoraxchirurgie_CABG en Klepchirurgie
        16671, --DMC_Thoraxchirurgie_CABG en Klepchirurgie
        13117, --D_Thoraxchirurgie_Cardio anders
        16672, --DMC_Thoraxchirurgie_Cardio anders
        13118, --D_Thoraxchirurgie_Aorta chirurgie
        16670, --DMC_Thoraxchirurgie_Aorta chirurgie
        13119, --D_Thoraxchirurgie_Pulmonale chirurgie
        16673, --DMC_Thoraxchirurgie_Pulmonale chirurgie

        13141, --D_Algemene chirurgie_Algemeen
        16642, --DMC_Algemene chirurgie_Algemeen
        13121, --D_Algemene chirurgie_Buikchirurgie
        16643, --DMC_Algemene chirurgie_Buikchirurgie
        13123, --D_Algemene chirurgie_Endocrinologische chirurgie
        16644, --DMC_Algemene chirurgie_Endocrinologische chirurgie
        13145, --D_Algemene chirurgie_KNO/Overige
        16645, --DMC_Algemene chirurgie_KNO/Overige
        13125, --D_Algemene chirurgie_Orthopedische chirurgie
        16646, --DMC_Algemene chirurgie_Orthopedische chirurgie
        13122, --D_Algemene chirurgie_Transplantatie chirurgie
        16647, --DMC_Algemene chirurgie_Transplantatie chirurgie
        13124, --D_Algemene chirurgie_Trauma
        16648, --DMC_Algemene chirurgie_Trauma
        13126, --D_Algemene chirurgie_Urogenitaal
        16649, --DMC_Algemene chirurgie_Urogenitaal
        13120, --D_Algemene chirurgie_Vaatchirurgie
        16650, --DMC_Algemene chirurgie_Vaatchirurgie

        13128, --D_Neurochirurgie _Vasculair chirurgisch
        16661, --DMC_Neurochirurgie _Vasculair chirurgisch
        13129, --D_Neurochirurgie _Tumor chirurgie
        16660, --DMC_Neurochirurgie _Tumor chirurgie
        13130, --D_Neurochirurgie_Overige
        16662, --DMC_Neurochirurgie_Overige

        18596, --Apache II Operatief  Gastr-intenstinaal
        18597, --Apache II Operatief Cardiovasculair
        18598, --Apache II Operatief Hematologisch
        18599, --Apache II Operatief Metabolisme
        18600, --Apache II Operatief Neurologisch
        18601, --Apache II Operatief Renaal
        18602, --Apache II Operatief Respiratoir

        17008, --APACHEIV Post-operative cardiovascular
        17009, --APACHEIV Post-operative gastro-intestinal
        17010, --APACHEIV Post-operative genitourinary
        17011, --APACHEIV Post-operative hematology
        17012, --APACHEIV Post-operative metabolic
        17013, --APACHEIV Post-operative musculoskeletal /skin
        17014, --APACHEIV Post-operative neurologic
        17015, --APACHEIV Post-operative respiratory
        17016, --APACHEIV Post-operative transplant
        17017, --APACHEIV Post-operative trauma

        --MEDICAL
        13133, --D_Interne Geneeskunde_Cardiovasculair
        16653, --DMC_Interne Geneeskunde_Cardiovasculair
        13134, --D_Interne Geneeskunde_Pulmonaal
        16658, --DMC_Interne Geneeskunde_Pulmonaal
        13135, --D_Interne Geneeskunde_Abdominaal
        16652, --DMC_Interne Geneeskunde_Abdominaal
        13136, --D_Interne Geneeskunde_Infectieziekten
        16655, --DMC_Interne Geneeskunde_Infectieziekten
        13137, --D_Interne Geneeskunde_Metabool
        16656, --DMC_Interne Geneeskunde_Metabool
        13138, --D_Interne Geneeskunde_Renaal
        16659, --DMC_Interne Geneeskunde_Renaal
        13139, --D_Interne Geneeskunde_Hematologisch
        16654, --DMC_Interne Geneeskunde_Hematologisch
        13140, --D_Interne Geneeskunde_Overige
        16657, --DMC_Interne Geneeskunde_Overige

        13131, --D_Neurologie_Vasculair neurologisch
        16664, --DMC_Neurologie_Vasculair neurologisch
        13132, --D_Neurologie_Overige
        16663, --DMC_Neurologie_Overige
        13127, --D_KNO/Overige

        18589, --Apache II Non-Operatief Cardiovasculair
        18590, --Apache II Non-Operatief Gastro-intestinaal
        18591, --Apache II Non-Operatief Hematologisch
        18592, --Apache II Non-Operatief Metabolisme
        18593, --Apache II Non-Operatief Neurologisch
        18594, --Apache II Non-Operatief Renaal
        18595, --Apache II Non-Operatief Respiratoir

        16998, --APACHE IV Non-operative cardiovascular
        16999, --APACHE IV Non-operative Gastro-intestinal
        17000, --APACHE IV Non-operative genitourinary
        17001, --APACHEIV  Non-operative haematological
        17002, --APACHEIV  Non-operative metabolic
        17003, --APACHEIV Non-operative musculo-skeletal
        17004, --APACHEIV Non-operative neurologic
        17005, --APACHEIV Non-operative respiratory
        17006, --APACHEIV Non-operative transplant
        17007, --APACHEIV Non-operative trauma

        --NICE: surgical/medical combined in same parameter
        18669, --NICE APACHEII diagnosen
        18671 --NICE APACHEIV diagnosen
    )
), sepsis AS (
    SELECT
        admissionid,
        CASE valueid
            WHEN 1 THEN 1 --'Ja'
            WHEN 2 THEN 0 --'Nee'
        END as sepsis_at_admission,
        ROW_NUMBER() OVER(
            PARTITION BY
                admissionid
            ORDER BY
                updatedat DESC) AS rownum --prioritizes diagnosis with last update time
    FROM listitems
    WHERE
        itemid = 15808
), sepsis_antibiotics AS ( --non prophylactic antibiotics
    SELECT
        admissionid,
        CASE
            WHEN COUNT(*) > 0 THEN 1
            ELSE 0
        END AS sepsis_antibiotics_bool,
        STRING_AGG(DISTINCT item, '; ') AS sepsis_antibiotics_given
    FROM drugitems
    WHERE
        itemid IN (
            6834, --Amikacine (Amukin)
            6847, --Amoxicilline (Clamoxyl/Flemoxin)
            6871, --Benzylpenicilline (Penicilline)
            6917, --Ceftazidim (Fortum)
            --6919, --Cefotaxim (Claforan) -> prophylaxis
            6948, --Ciprofloxacine (Ciproxin)
            6953, --Rifampicine (Rifadin)
            6958, --Clindamycine (Dalacin)
            7044, --Tobramycine (Obracin)
            --7064, --Vancomycine -> prophylaxis for valve surgery
            7123, --Imipenem (Tienam)
            7185, --Doxycycline (Vibramycine)
            --7187, --Metronidazol (Flagyl) -> often used for GI surgical prophylaxis
            --7208, --Erythromycine (Erythrocine) -> often used for gastroparesis
            7227, --Flucloxacilline (Stafoxil/Floxapen)
            7231, --Fluconazol (Diflucan)
            7232, --Ganciclovir (Cymevene)
            7233, --Flucytosine (Ancotil)
            7235, --Gentamicine (Garamycin)
            7243, --Foscarnet trinatrium (Foscavir)
            7450, --Amfotericine B (Fungizone)
            --7504, --X nader te bepalen --non-stock medication
            8127, --Meropenem (Meronem)
            8229, --Myambutol (ethambutol)
            8374, --Kinine dihydrocloride
            --8375, --Immunoglobuline (Nanogam) -> not anbiotic
            --8394, --Co-Trimoxazol (Bactrimel) -> often prophylactic (unless high dose)
            8547, --Voriconazol(VFEND)
            --9029, --Amoxicilline/Clavulaanzuur (Augmentin) -> often used for ENT surgical prophylaxis
            9030, --Aztreonam (Azactam)
            9047, --Chlooramfenicol
            --9075, --Fusidinezuur (Fucidin) -> prophylaxis
            9128, --Piperacilline (Pipcil)
            9133, --Ceftriaxon (Rocephin)
            --9151, --Cefuroxim (Zinacef) -> often used for GI/transplant surgical prophylaxis
            --9152, --Cefazoline (Kefzol) -> prophylaxis for cardiac surgery
            9458, --Caspofungine
            9542, --Itraconazol (Trisporal)
            --9602, --Tetanusimmunoglobuline -> prophylaxis/not antibiotic
            12398, --Levofloxacine (Tavanic)
            12772, --Amfotericine B lipidencomplex  (Abelcet)
            15739, --Ecalta (Anidulafungine)
            16367, --Research Anidulafungin/placebo
            16368, --Research Caspofungin/placebo
            18675, --Amfotericine B in liposomen (Ambisome )
            19137, --Linezolid (Zyvoxid)
            19764, --Tigecycline (Tygacil)
            19773, --Daptomycine (Cubicin)
            20175 --Colistine
        )
        AND start < 24*60*60*1000 --within 24 hours (to correct for antibiotics administered before ICU)
    GROUP BY admissionid
), other_antibiotics AS ( --'prophylactic' antibiotics that may be used for sepsis
    SELECT
        admissionid,
        CASE
            WHEN COUNT(*) > 0 THEN 1
            ELSE 0
        END AS other_antibiotics_bool,
        STRING_AGG(DISTINCT item, '; ') AS other_antibiotics_given
    FROM drugitems
    WHERE
        itemid IN (
            7064, --Vancomycine -> prophylaxis for valve surgery
            7187, --Metronidazol (Flagyl) -> often used for GI surgical prophylaxis
            8394, --Co-Trimoxazol (Bactrimel) -> often prophylactic (unless high dose)
            9029, --Amoxicilline/Clavulaanzuur (Augmentin) -> often used for ENT surgical prophylaxis
            9151, --Cefuroxim (Zinacef) -> often used for GI surgical prophylaxis
            9152 --Cefazoline (Kefzol) -> prophylaxis
        )
        AND start < 24*60*60*1000 --within 24 hours (to correct for antibiotics administered before ICU)
    GROUP BY admissionid
), cultures AS (
    SELECT
        admissionid,
        CASE
            WHEN COUNT(*) > 0 THEN 1
            ELSE 0
        END AS sepsis_cultures_bool,
        STRING_AGG(DISTINCT item, '; ') AS sepsis_cultures_drawn
    FROM procedureorderitems
    WHERE
        itemid IN (
        --8097, --Sputumkweek afnemen -> often used routinely
        --8418, --Urinekweek afnemen
        --8588, --MRSA kweken afnemen
        9189, --Bloedkweken afnemen
        9190, --Cathetertipkweek afnemen
        --9191, --Drainvochtkweek afnemen
        --9192, --Faeceskweek afnemen -> Clostridium
        --9193, --X-Kweek nader te bepalen
        --9194, --Liquorkweek afnemen
        --9195, --Neuskweek afnemen
        --9197, --Perineumkweek afnemen -> often used routinely
        -9198, --Rectumkweek afnemen -> often used routinely
        9200, --Wondkweek afnemen
        9202, --Ascitesvochtkweek afnemen
        --9203, --Keelkweek afnemen -> often used routinely
        --9204, --SDD-kweken afnemen -> often used routinely
        9205 --Legionella sneltest (urine)
        --1302, --SDD Inventarisatiekweken afnemen -> often used routinely
        --19663, --Research Neuskweek COUrSe
        --19664, --Research Sputumkweek COUrSe
        )
        AND registeredat < 6*60*60*1000 --within 6 hours
    GROUP BY admissionid
)
SELECT
    admissions.*
    , diagnosis_type
    , diagnosis, diagnosis_id
    , diagnosis_subgroup
    , diagnosis_subgroup_id
    , diagnosis_group
    , diagnosis_group_id
    , surgical
    , sepsis_at_admission
    , sepsis_antibiotics_bool
    , sepsis_antibiotics_given
    , other_antibiotics_bool
    , other_antibiotics_given
    , sepsis_cultures_bool
    , sepsis_cultures_drawn
FROM admissions
LEFT JOIN diagnoses on admissions.admissionid = diagnoses.admissionid
LEFT JOIN diagnosis_subgroups on admissions.admissionid = diagnosis_subgroups.admissionid
LEFT JOIN diagnosis_groups on admissions.admissionid = diagnosis_groups.admissionid
LEFT JOIN sepsis on admissions.admissionid = sepsis.admissionid
LEFT JOIN sepsis_antibiotics on admissions.admissionid = sepsis_antibiotics.admissionid
LEFT JOIN other_antibiotics on admissions.admissionid = other_antibiotics.admissionid
LEFT JOIN cultures on admissions.admissionid = cultures.admissionid
WHERE --only diagnosis with last update time
    (diagnoses.rownum = 1 OR diagnoses.rownum IS NULL) AND
    (diagnosis_subgroups.rownum = 1 OR diagnosis_subgroups.rownum IS NULL) AND
    (diagnosis_groups.rownum = 1 OR diagnosis_groups.rownum IS NULL) AND
    (sepsis.rownum = 1 OR sepsis.rownum IS NULL)
;
```

,patientid,admissionid,admissioncount,location,urgency,origin,admittedat,admissionyeargroup,dischargedat,lengthofstay,destination,gender,agegroup,dateofdeath,weightgroup,weightsource,heightgroup,heightsource,specialty,diagnosis_type,diagnosis,diagnosis_id,diagnosis_subgroup,diagnosis_subgroup_id,diagnosis_group,diagnosis_group_id,surgical,sepsis_at_admission,sepsis_antibiotics_bool,sepsis_antibiotics_given,other_antibiotics_bool,other_antibiotics_given,sepsis_cultures_bool,sepsis_cultures_drawn
0,0,0,1,IC,0,None,0,2003-2009,148800000,42,16,Vrouw,80+,NaN,60-69,Anamnestisch,160-169,Anamnestisch,Cardiochirurgie,Legacy ICU,CABG,1.0,CABG en Klepchirurgie,1.0,Thoraxchirurgie,1.0,1.0,NaN,NaN,None,1.0,Cefazoline (Kefzol),NaN,None
1,1,1,1,IC,0,None,0,2010-2016,96120000,26,15,Man,60-69,NaN,70-79,Anamnestisch,170-179,Anamnestisch,Cardiochirurgie,Legacy ICU,Bentall,5.0,Aorta chirurgie,3.0,Thoraxchirurgie,1.0,1.0,0.0,NaN,None,1.0,Cefazoline (Kefzol); Vancomycine,NaN,None
2,2,2,1,IC,1,Eerste Hulp afdeling zelfde ziekenhuis,0,2010-2016,84240000,23,15,Man,60-69,NaN,90-99,Anamnestisch,180-189,Anamnestisch,Cardiochirurgie,NICE APACHE IV,CABG with aortic valve replacement,237.0,CABG en Klepchirurgie,1.0,Operatief Cardiovasculair,1.0,1.0,0.0,NaN,None,1.0,Cefazoline (Kefzol); Vancomycine,NaN,None
3,3,3,1,IC,0,None,0,2003-2009,84900000,23,14,Man,50-59,NaN,90-99,None,180-189,Gemeten,Cardiochirurgie,None,None,NaN,None,NaN,None,NaN,NaN,NaN,NaN,None,1.0,Cefazoline (Kefzol),NaN,None
4,4,4,1,IC&MC,0,Verpleegafdeling zelfde ziekenhuis,0,2010-2016,180900000,50,19,Man,70-79,NaN,70-79,Anamnestisch,170-179,Anamnestisch,Cardiochirurgie,NICE APACHE IV,"CABG alone, coronary artery bypass grafting",233.0,CABG en Klepchirurgie,1.0,Operatief Cardiovasculair,1.0,1.0,0.0,NaN,None,1.0,Cefazoline (Kefzol),NaN,None


In [7]:
#surgical
surgical_total = len(combined_diagnoses[combined_diagnoses['surgical'] == 1])
surgical_pct_total = 100*surgical_total/no_admissions_total

surgical_ICU = len(combined_diagnoses[(combined_diagnoses['surgical'] == 1)  & combined_diagnoses['location'].str.contains('IC')])
surgical_pct_ICU = 100*surgical_ICU/no_admissions_ICU

surgical_MCU = len(combined_diagnoses[(combined_diagnoses['surgical'] == 1)  & combined_diagnoses['location'].str.contains('^MC$')])
surgical_pct_MCU = 100*surgical_MCU/no_admissions_MCU

#medical
medical_total = len(combined_diagnoses[combined_diagnoses['surgical'] == 0])
medical_pct_total = 100*medical_total/no_admissions_total

medical_ICU = len(combined_diagnoses[(combined_diagnoses['surgical'] == 0)  & combined_diagnoses['location'].str.contains('IC')])
medical_pct_ICU = 100*medical_ICU/no_admissions_ICU

medical_MCU = len(combined_diagnoses[(combined_diagnoses['surgical'] == 0)  & combined_diagnoses['location'].str.contains('^MC$')])
medical_pct_MCU = 100*medical_MCU/no_admissions_MCU

#surgical/medical unknown
medical_surgical_na_total = len(combined_diagnoses[pd.isnull(combined_diagnoses['surgical'])])
medical_surgical_na_pct_total = 100*medical_surgical_na_total/no_admissions_total

medical_surgical_na_ICU = len(combined_diagnoses[pd.isnull(combined_diagnoses['surgical']) & combined_diagnoses['location'].str.contains('IC')])
medical_surgical_na_pct_ICU = 100*medical_surgical_na_ICU/no_admissions_ICU

medical_surgical_na_MCU = len(combined_diagnoses[pd.isnull(combined_diagnoses['surgical']) & combined_diagnoses['location'].str.contains('^MC$')])
medical_surgical_na_pct_MCU = 100*medical_surgical_na_MCU/no_admissions_MCU

#urgency
urgency_total = len(combined_diagnoses[combined_diagnoses['urgency'] == '1']) #apparently the postgresql bit become an object
urgency_pct_total = 100*urgency_total/no_admissions_total

urgency_ICU = len(combined_diagnoses[(combined_diagnoses['urgency'] == '1')  & combined_diagnoses['location'].str.contains('IC')])
urgency_pct_ICU = 100*urgency_ICU/no_admissions_ICU

urgency_MCU = len(combined_diagnoses[(combined_diagnoses['urgency'] == '1')  & combined_diagnoses['location'].str.contains('^MC$')])
urgency_pct_MCU = 100*urgency_MCU/no_admissions_MCU

## Reason for admission based on Legacy/APACHE II or IV diagnosis

## Admission diagnoses
Uses regular expression pattern matching for classification. Alternative approach: manually assign diagnostic codes to groups.

In [8]:
import warnings
warnings.filterwarnings("ignore", 'This pattern has match groups')

### Surgical

In [9]:
re_cardiosurg = r'(CABG|AVR|hartchirurgie|heart surgery|Chron. cardiovasculaire ziekte|hartkleppen' + \
r'|cardiovascula|MVP|MVR|mitral|tricuspid|pericard|aortic.*valve|lobectom|segment|thorax|Bentall|aorta-ascendens|aorta-boog' + \
r'|aorta-wortel|aorta-descendens|lung|pneumectomie|bullectom|respiratoir neoplasm|thoracoscop|thoracotom(y|ie)|respirato' + \
r'|vrije wand ruptuur|VSR|ASD|pleurectom|intracardiac|aneurysmectom|congenital defect repair)(?! for esophag)'
re_neurosurg = r'neuro|tentorieel|cranial|subdur|epidur|subarachn|intracerbr|hoofdtrauma|SAB|S.A.H.|CNS|Hoofd|seizures' + \
r'|craniotom|cranioplast|spinal|dwarslaesie|ventriculstom|transphenoidal|brain|A.V.M.|Arteriovenous malformation' + \
r'|brughoek|shunts and revisions|stereotactic|Burr hole|cerebrospinal'
re_vascsurg = r'vaatchirurgie|vasc.*surg|EVAR|aorta vervanging|perifeer vasc|embolectom|aneurysm|carotid|endovasc' + \
r'|dissectie|endarterectom|thrombectomy|dilatation|PTCA|all other bypass|femoral-popliteal|aorto-femoral|femoral-femoral' + \
r'|transplantectom'
re_gisurg = r'oesophagus|esophageal|maag|stomach|darm|intestin|gastro-intestin|pancreatitis|laparotom|gastro-intestinale perforatie' + \
r'|galblaas|Bleeding-.*GI|other GI|colon|rectal|GI.*surgery|GI obstruction|Whipple|diverticular|appendectomy' + \
r'|peritonitis|cholecystectomy|exenteration'
re_uro = r'(?<!ne)(urolog|cystectomy|genitourinary surgery|prostatectom|ileal\-conduit|orchiectomy|bladder repair' + \
r'|nefrectom|nephrectom|renaal neopsplama)'
re_obgyn = r'hysterectom|Cesarean|ectopic pregnancy'
re_gensurg = r'mond/keel|cancer oral|cancer laryngeal|spondylodes|Fusion-spinal|devices for spine|orthopedic' + \
r'|renaal|metabol|endocrin|thyroid|hip replacement|knee replacement|adrenalectom|tracheostomy|bewaking|amputation|apnea-sleep' + \
r'|lymph node dissection|mastectomy|cosmetic|fracture-pathological|bewaking'
re_trauma_surg = r'(?<!non-)(?<!see )(trauma|hypotherm|smoke inhalation)(?!, see trauma)(?! see)(?!: see)'
re_tx_surg = r'niertransplantatie|kidney transplant|renaal transplantatie|pancreastransplantatie'

re_respfailure_surg = r'resp.*insuff|na respiratoir arrest|arrest, respiratory|atelectas|pneumoni|na ok'

re_sepsis_surg = r'sepsis|pneumoni|GI perforation|perforation/rupture|infection|abscess|GI Vascular ischemia|diverticular|appendectomy|peritonitis'
re_cardiacarrest_surg = r'Cardiac arrest.*respiratory arrest|na reanimatie'
re_hepa = r'lever'
re_surg_other = r'diagnose anders|respiratoir|cardiovasculair|niet operatief'

re_surgical_medical = r'Bloeding tractus digestivus|Haemorragische shock|Gastro-intestinale bloeding|Bleeding, upper GI' + \
r'|hematologisch|Hematologische maligniteit|Haematologisch|Renaal'

re_surgical = '(' + \
    re_cardiosurg + '|' + \
    re_neurosurg + '|' + \
    re_vascsurg + '|' + \
    re_gisurg + '|' + \
    re_uro + '|' + \
    re_obgyn + '|' + \
    re_gensurg + '|' + \
    re_trauma_surg + '|' + \
    re_tx_surg + '|' + \
    re_hepa + '|' + \
    re_surg_other + '|' + \
    re_surgical_medical + '|' + \
    re_respfailure_surg + '|' + \
    re_sepsis_surg + '|' + \
    re_cardiacarrest_surg + '|' + \
    ')'

#check to see if we caught all (should return empty data frame):
combined_diagnoses[(combined_diagnoses['surgical'] == 1
) & ~(combined_diagnoses['diagnosis'].str.contains(
    re_surgical, na=False, flags=re.IGNORECASE))].groupby(['diagnosis_group','diagnosis']).size().to_frame('number of admissions').sort_values('number of admissions', ascending=False).reset_index()

,diagnosis_group,diagnosis,number of admissions


## medical

In [10]:
re_respfailure_med = r'(?<! without )(resp.*insuff|pneumoni|respirato|luchtweg obstructie|obstruction-airway' + \
r'|chronisch obstructieve longziekte|emphysema|asthma|aspiratie|aspiration|longembolie|pulmonary|pulmonaire' + \
r'|long|lung|atelectas|ALI|ARDS|pleural|cancer, lung|pneumothorax|verdrinking|near drowning|weaning|hemothorax|PCP)'
re_cardiacarrest_med = r'Cardiac arrest.*respiratory arrest|na reanimatie'
re_sepsis_med = r'sepsis|septic|infect|pneumoni|cholangitis|pancr|endocarditis|meningitis|GI perforation|abces|abscess|darm ischaemie|GI vascular|fasciitis' + \
r'|inflammatory|peritonitis'
re_cardio = r'cardiogene shock|shock, cardiogenic|angina|ritme|rhythm|cardiovascular|cardiovasculair' + \
r'|myocardial|endocarditis|coronair|coronary|cardiomyopath|tamponade|pericardial|papillary muscle|^MI|hartkleppen' + \
r'|CHF|hart falen|decompensatio cordis'
re_neuro = '(?<!see )(insult|seizure|CVA|observatie neurologische status|intracerebraal haematoom|intracranial|intracerebr|subdur|subarachno' + \
r'|epidur|coma|neurologisch|neurologic|CZS|S\.A\.B\.|neurologie|hoofdtrauma|head|neuro|muscula|spinal|meningitis|encephalitis' + \
r'|myasthenia|vaatspasme protocol|Guillian-Barre|encephalopath|musculoskeletal)(?!-see Neurological System)'
re_bleeding = r'bloeding tractus digestivus|gastro-intestinale bloeding|gastro-intestinaal|bleeding, upper GI|bleeding, lower GI' + \
r'|bleeding, GI|ulcer'
re_gi_med = r'ileus|GI medical|GI obstruction'
re_tox = r'intox|overdosis|overdose|toxicity|withdrawal|drug'
re_trauma_med = r'(?<!non-)(?<!see )(trauma|hypotherm|smoke inhalation)(?!, see trauma)(?! see)(?!: see)'
re_hemonc = r'malign|hematolog|cancer|bloeding|TTP|HUS|leukemi|pancytopen|coagulopath|anemia|neutropen|lymph|sickle'
re_endo_med = r'metabolisme|keto-acidose|diabetic|metabolic|endocrine|hypertens|acid-base|hypoglyc|thyroid'
re_shock_med = r'hypovolemische shock|shock|haemorr|hemorrha|anaphylaxis'
re_nefro_med = r'renaal|renal|tubulus|genitourinary|urolog|nefr'
re_hepa_med = r'lever|hepatic'
re_obgyn_med = r'obstetrie|postpartum|eclampsia'
re_vasc_med = r'aneurysm|vascular|cardiovascular medical|Thrombus,arterial|vascular medical|dissect|EVAR|embolectom'
re_mon_med = r'apnea, sleep|monitoring|bewaking|observatie'
re_tx_med = r'transplant'

re_medical = '(' + \
re_respfailure_med + '|' + \
re_cardiacarrest_med + '|' + \
re_sepsis_med + '|' + \
re_cardio + '|' + \
re_neuro + '|' + \
re_bleeding + '|' + \
re_gi_med + '|' + \
re_tox + '|' + \
re_trauma_med + '|' + \
re_hemonc + '|' + \
re_endo_med + '|' + \
re_shock_med + '|' + \
re_nefro_med + '|' + \
re_hepa_med + '|' + \
re_obgyn_med + '|' + \
re_vasc_med + '|' + \
re_mon_med + '|' + \
re_tx_med + '|' + \
')'

#check to see if we caught all (should return empty data frame):
combined_diagnoses[(combined_diagnoses['surgical'] == 0
) & ~(combined_diagnoses['diagnosis'].str.contains(
    re_medical, na=False, flags=re.IGNORECASE))].groupby(['diagnosis_group','diagnosis']).size().to_frame('number of admissions').sort_values('number of admissions', ascending=False).reset_index()

,diagnosis_group,diagnosis,number of admissions


### Cardiothoracic surgery

In [11]:
cardiosurg = combined_diagnoses[
    (combined_diagnoses['surgical'] == 1) 
    & (combined_diagnoses['diagnosis'].str.contains(re_cardiosurg, na=False, flags=re.IGNORECASE))]

cardiosurg_counts = cardiosurg.groupby([
    'diagnosis_group','diagnosis']).size().to_frame('n_admissions').sort_values(
    'n_admissions', ascending=False).reset_index()
cardiosurg_counts.head()

,diagnosis_group,diagnosis,n_admissions
0,Thoraxchirurgie,CABG,1389
1,Post-operative cardiovascular,"CABG alone, coronary artery bypass grafting",643
2,Operatief Cardiovasculair,"CABG alone, coronary artery bypass grafting",642
3,Thoraxchirurgie,AVR,458
4,Thoraxchirurgie,Gecombineerde hartchirurgie,366


In [12]:
cardiosurg_counts['n_admissions'].sum()

5939

### Respiratory failure

In [13]:
respfailure = combined_diagnoses[(
    (combined_diagnoses['surgical'] == 1)
    & (combined_diagnoses['diagnosis'].str.contains(re_respfailure_surg, na=False, flags=re.IGNORECASE))
) | (
    (combined_diagnoses['surgical'] == 0)
    & (combined_diagnoses['diagnosis'].str.contains(re_respfailure_med, na=False, flags=re.IGNORECASE))
)]

respfailure_counts = respfailure.groupby([
    'diagnosis_group','diagnosis']).size().to_frame('n_admissions').sort_values(
    'n_admissions', ascending=False).reset_index()
respfailure_counts.head()

,diagnosis_group,diagnosis,n_admissions
0,Interne geneeskunde,Respiratoire insufficiëntie,326
1,Interne geneeskunde,Pneumonie,146
2,Non-operative respiratory,"Pneumonia, bacterial",75
3,Non-operatief Respiratoir,"Pneumonia, bacterial",65
4,Interne geneeskunde,"Pneumonia, bacterial",57


In [14]:
respfailure_counts['n_admissions'].sum()

1568

### Neurosurgery

In [15]:
neurosurg = combined_diagnoses[
    (combined_diagnoses['surgical'] == 1) 
    & (combined_diagnoses['diagnosis'].str.contains(re_neurosurg, na=False, flags=re.IGNORECASE))
]
neurosurg_counts = neurosurg.groupby([
    'diagnosis_group','diagnosis']).size().to_frame('n_admissions').sort_values(
    'n_admissions', ascending=False).reset_index()
neurosurg_counts.head()

,diagnosis_group,diagnosis,n_admissions
0,Neurochirurgie,Neurochirurgisch Trauma,174
1,Neurochirurgie,TC supra tentorieel,167
2,Post-operative neurologic,"Neoplasm-cranial, surgery for (excluding transphenoidal)",113
3,Operatief Neurologisch,"Neoplasm-cranial, surgery for (excluding transphenoidal)",112
4,Neurochirurgie,Overige neurologie/neurochirurgie,106


In [16]:
neurosurg_counts['n_admissions'].sum()

1617

### Gastro-intestinal surgery

In [17]:
gisurg = combined_diagnoses[
    (combined_diagnoses['surgical'] == 1) 
    & (combined_diagnoses['diagnosis'].str.contains(re_gisurg, na=False, flags=re.IGNORECASE))
]
gisurg_counts = gisurg.groupby([
    'diagnosis_group','diagnosis']).size().to_frame('n_admissions').sort_values(
    'n_admissions', ascending=False).reset_index()
gisurg_counts.head()

,diagnosis_group,diagnosis,n_admissions
0,Algemene chirurgie,Oesophagus/maag chirurgie,171
1,Algemene chirurgie,Dunne/dikkedarm chirurgie (perforatie of obstructie),169
2,Algemene chirurgie,Gastro-intestinaal voor neoplasma,55
3,Post-operative gastro-intestinal,"Cancer-esophageal, surgery for (abdominal approach)",43
4,Operatief Gastro-Intestinaal,"Cancer-esophageal, surgery for (abdominal approach)",43


In [18]:
gisurg_counts['n_admissions'].sum()

1149

### Cardiac arrest

In [19]:
cardiacarrest = combined_diagnoses[
    (
        (combined_diagnoses['surgical'] == 1)
        & (combined_diagnoses['diagnosis'].str.contains(re_cardiacarrest_surg, na=False, flags=re.IGNORECASE))
    ) | (
        (combined_diagnoses['surgical'] == 0)
        & (combined_diagnoses['diagnosis'].str.contains(re_cardiacarrest_med, na=False, flags=re.IGNORECASE))
    )
]
cardiacarrest_counts = cardiacarrest.groupby([
    'diagnosis_group','diagnosis']).size().to_frame('n_admissions').sort_values(
    'n_admissions', ascending=False).reset_index()
cardiacarrest_counts.head()

,diagnosis_group,diagnosis,n_admissions
0,Interne geneeskunde,Na reanimatie,437
1,Non-operative cardiovascular,Cardiac arrest (with or without respiratory arrest;,181
2,Non-operatief Cardiovasculair,Cardiac arrest (with or without respiratory arrest;,173
3,Interne geneeskunde,Cardiac arrest (with or without respiratory arrest;,155
4,Algemene chirurgie,Cardiac arrest (with or without respiratory arrest; for respiratory arrest see,4


In [20]:
cardiacarrest_counts['n_admissions'].sum()

961

### Vascular surgery

In [21]:
vascsurg = combined_diagnoses[
    (combined_diagnoses['surgical'] == 1) 
    & (combined_diagnoses['diagnosis'].str.contains(re_vascsurg, na=False, flags=re.IGNORECASE))
    #exclude cerebral aneurysms:
    & ~(combined_diagnoses['diagnosis_group'].str.contains('neuro', na=False, flags=re.IGNORECASE)) 
]
vascsurg_counts = vascsurg.groupby([
    'diagnosis_group','diagnosis']).size().to_frame('n_admissions').sort_values(
    'n_admissions', ascending=False).reset_index()
vascsurg_counts.head()

,diagnosis_group,diagnosis,n_admissions
0,Algemene chirurgie,Vaatchirurgie abdominaal,212
1,Algemene chirurgie,Vaatchirurgie perifeer,62
2,Algemene chirurgie,EVAR,61
3,Algemene chirurgie,Thoraco-abdominale aorta vervanging,54
4,Operatief Cardiovasculair,"Aneurysm, thoracic aortic",39


In [22]:
vascsurg_counts['n_admissions'].sum()

1038

### Trauma

In [23]:
trauma = combined_diagnoses[
    (
        (combined_diagnoses['surgical'] == 1)
        & (combined_diagnoses['diagnosis'].str.contains(re_trauma_surg, na=False, flags=re.IGNORECASE))
    ) | (
        (combined_diagnoses['surgical'] == 0)
        & (combined_diagnoses['diagnosis'].str.contains(re_trauma_med, na=False, flags=re.IGNORECASE))
    )
]

trauma_counts = trauma.groupby(['diagnosis_group','diagnosis']).size().to_frame('n_admissions').sort_values(
    'n_admissions', ascending=False).reset_index()
trauma_counts.head()

,diagnosis_group,diagnosis,n_admissions
0,Neurochirurgie,Neurochirurgisch Trauma,174
1,Algemene chirurgie,Multi trauma (operatief),102
2,Algemene chirurgie,Multi trauma (niet operatief),82
3,Algemene chirurgie,Thoraxtrauma (niet operatief),36
4,Algemene chirurgie,Hoofdtrauma (niet operatief),23


In [24]:
trauma_counts['n_admissions'].sum()

1241

### Sepsis

In [25]:
sepsis = combined_diagnoses[
    (
        (
            #use reasons for admission
            #surgical admissions with sepsis
            (combined_diagnoses['surgical'] == 1) 
            & (combined_diagnoses['diagnosis'].str.contains(re_sepsis_surg, na=False, flags=re.IGNORECASE))
        ) | (
            #medical admissions with sepsis
            (combined_diagnoses['surgical'] == 0) 
            & (combined_diagnoses['diagnosis'].str.contains(re_sepsis_med, na=False, flags=re.IGNORECASE))
        ) | (
            #uses documentation at admission form (Early Goal Directed Therapy)
            (combined_diagnoses['sepsis_at_admission'] == 1)
        ) | (
            #uses administered (therapeutic) antibiotics for determining sepsis
            (combined_diagnoses['sepsis_antibiotics_bool'] == 1)
        ) | (
            #uses combination of administered antibiotics (that sometimes are used as prophylaxis) AND 
            #drawn cultures for determining sepsis
            (combined_diagnoses['other_antibiotics_bool'] == 1) 
            & (combined_diagnoses['sepsis_cultures_bool'] == 1)
        )
    ) & ~(combined_diagnoses['sepsis_at_admission'] == 0) 
    #exclude all diagnoses where explicitly 'no sepsis' was documented
]
sepsis_counts = sepsis.fillna('N/A').groupby(['diagnosis_group','diagnosis']).size().to_frame('n_admissions').sort_values(
    'n_admissions', ascending=False).reset_index()
sepsis_counts.head()

,diagnosis_group,diagnosis,n_admissions
0,N/A,N/A,1104
1,Interne geneeskunde,Pneumonie,126
2,Algemene chirurgie,Dunne/dikkedarm chirurgie (perforatie of obstructie),117
3,Interne geneeskunde,Respiratoire insufficiëntie,112
4,Interne geneeskunde,Infectieuze aandoening overig,83


In [26]:
sepsis_counts['n_admissions'].sum()

3152

### Neurologic disorders

In [27]:
neuro = combined_diagnoses[
    (combined_diagnoses['surgical'] == 0) 
    & (combined_diagnoses['diagnosis'].str.contains(re_neuro, na=False, flags=re.IGNORECASE))
    #exclude trauma cases
    & ~(combined_diagnoses['diagnosis'].str.contains(re_trauma_surg, na=False, flags=re.IGNORECASE))
]

neuro_counts = neuro.groupby([
    'diagnosis_group','diagnosis']).size().to_frame('n_admissions').sort_values(
    'n_admissions', ascending=False).reset_index()
neuro_counts.head()

,diagnosis_group,diagnosis,n_admissions
0,Neurologie,Epileptisch insult,58
1,Neurologie,Observatie neurologische status,35
2,Neurologie,Intracerebraal haematoom,30
3,Neurologie,Coma,29
4,Non-operatief Neurologisch,Seizures (primary-no structural brain disease),29


In [28]:
neuro_counts['n_admissions'].sum()

627

### Cardiac disorders

In [29]:
cardio = combined_diagnoses[
    (combined_diagnoses['surgical'] == 0) 
    & (combined_diagnoses['diagnosis'].str.contains(re_cardio, na=False, flags=re.IGNORECASE))
]
cardio_counts = cardio.groupby([
    'diagnosis_group','diagnosis']).size().to_frame('n_admissions').sort_values(
    'n_admissions', ascending=False).reset_index()
cardio_counts.head()

,diagnosis_group,diagnosis,n_admissions
0,Interne geneeskunde,Decompensatio cordis,87
1,Interne geneeskunde,Cardiogene shock,69
2,Non-operatief Cardiovasculair,"CHF, congestive heart failure",55
3,Non-operative cardiovascular,"CHF, congestive heart failure",52
4,Interne geneeskunde,"CHF, congestive heart failure",30


In [30]:
cardio_counts['n_admissions'].sum()

537

In [31]:
#reason for admission

## cardiac surgery
reason_car_surg_total = len(cardiosurg)
reason_car_surg_pct_total = 100*reason_car_surg_total/no_admissions_total
reason_car_surg_ICU = len(cardiosurg[cardiosurg['location'].str.contains('IC')])
reason_car_surg_pct_ICU = 100*reason_car_surg_ICU/no_admissions_ICU
reason_car_surg_MCU = len(cardiosurg[cardiosurg['location'].str.contains('^MC$')])
reason_car_surg_pct_MCU = 100*reason_car_surg_MCU/no_admissions_MCU

## neurosurgery
reason_neu_surg_total = len(neurosurg)
reason_neu_surg_pct_total =  100*reason_neu_surg_total/no_admissions_total
reason_neu_surg_ICU = len(neurosurg[neurosurg['location'].str.contains('IC')])
reason_neu_surg_pct_ICU =  100*reason_neu_surg_ICU/no_admissions_ICU
reason_neu_surg_MCU = len(neurosurg[neurosurg['location'].str.contains('^MC$')])
reason_neu_surg_pct_MCU =  100*reason_neu_surg_MCU/no_admissions_MCU

## gastro-intestinal surgery
reason_gi_surg_total = len(gisurg)
reason_gi_surg_pct_total = 100*reason_gi_surg_total/no_admissions_total
reason_gi_surg_ICU = len(gisurg[gisurg['location'].str.contains('IC')])
reason_gi_surg_pct_ICU = 100*reason_gi_surg_ICU/no_admissions_ICU
reason_gi_surg_MCU = len(gisurg[gisurg['location'].str.contains('^MC$')])
reason_gi_surg_pct_MCU = 100*reason_gi_surg_MCU/no_admissions_MCU

## cardiac arrest
reason_car_arrest_total = len(cardiacarrest)
reason_car_arrest_pct_total = 100*reason_car_arrest_total/no_admissions_total
reason_car_arrest_ICU = len(cardiacarrest[cardiacarrest['location'].str.contains('IC')])
reason_car_arrest_pct_ICU = 100*reason_car_arrest_ICU/no_admissions_ICU
reason_car_arrest_MCU = len(cardiacarrest[cardiacarrest['location'].str.contains('^MC$')])
reason_car_arrest_pct_MCU = 100*reason_car_arrest_MCU/no_admissions_MCU

## vascular surgery
reason_vasc_surg_total = len(vascsurg)
reason_vasc_surg_pct_total = 100*reason_vasc_surg_total/no_admissions_total
reason_vasc_surg_ICU = len(vascsurg[vascsurg['location'].str.contains('IC')])
reason_vasc_surg_pct_ICU = 100*reason_vasc_surg_ICU/no_admissions_ICU
reason_vasc_surg_MCU = len(vascsurg[vascsurg['location'].str.contains('^MC$')])
reason_vasc_surg_pct_MCU = 100*reason_vasc_surg_MCU/no_admissions_MCU

## trauma
reason_trauma_total = len(trauma)
reason_trauma_pct_total = 100*reason_trauma_total/no_admissions_total
reason_trauma_ICU = len(trauma[trauma['location'].str.contains('IC')])
reason_trauma_pct_ICU = 100*reason_trauma_ICU/no_admissions_ICU
reason_trauma_MCU = len(trauma[trauma['location'].str.contains('^MC$')])
reason_trauma_pct_MCU = 100*reason_trauma_MCU/no_admissions_MCU

## sepsis
reason_sepsis_total = len(sepsis)
reason_sepsis_pct_total = 100*reason_sepsis_total/no_admissions_total
reason_sepsis_ICU = len(sepsis[sepsis['location'].str.contains('IC')])
reason_sepsis_pct_ICU = 100*reason_sepsis_ICU/no_admissions_ICU
reason_sepsis_MCU = len(sepsis[sepsis['location'].str.contains('^MC$')])
reason_sepsis_pct_MCU = 100*reason_sepsis_MCU/no_admissions_MCU

## respiratory failure
reason_resp_failure_total = len(respfailure)
reason_resp_failure_pct_total = 100*reason_resp_failure_total/no_admissions_total
reason_resp_failure_ICU = len(respfailure[respfailure['location'].str.contains('IC')])
reason_resp_failure_pct_ICU = 100*reason_resp_failure_ICU/no_admissions_ICU
reason_resp_failure_MCU = len(respfailure[respfailure['location'].str.contains('^MC$')])
reason_resp_failure_pct_MCU = 100*reason_resp_failure_MCU/no_admissions_MCU

## cardio
reason_cardio_total = len(cardio)
reason_cardio_pct_total = 100*reason_cardio_total/no_admissions_total
reason_cardio_ICU = len(cardio[cardio['location'].str.contains('IC')])
reason_cardio_pct_ICU = 100*reason_cardio_ICU/no_admissions_ICU
reason_cardio_MCU = len(cardio[cardio['location'].str.contains('^MC$')])
reason_cardio_pct_MCU = 100*reason_cardio_MCU/no_admissions_MCU

## neuro
reason_neuro_total = len(neuro)
reason_neuro_pct_total = 100*reason_neuro_total/no_admissions_total
reason_neuro_ICU = len(neuro[neuro['location'].str.contains('IC')])
reason_neuro_pct_ICU = 100*reason_neuro_ICU/no_admissions_ICU
reason_neuro_MCU = len(neuro[neuro['location'].str.contains('^MC$')])
reason_neuro_pct_MCU = 100*reason_neuro_MCU/no_admissions_MCU

## Create a table in html

In [32]:
display(Markdown("""
<table>
    <thead>
        <tr>
            <th style="text-align:left;">Patient Characteristics</th>
            <th style="text-align:right;">Total</th>
            <th style="text-align:right;">ICU</th>
            <th style="text-align:right;">MCU</th>
        </tr>
        <tr>
    </thead>
    <tbody>
        <tr>
            <td style="text-align:left">Distinct patients, no.</td>
            <td>{no_patients_total}</td>
            <td>{no_patients_ICU}</td>
            <td>{no_patients_MCU}</td>
        </tr>    
        <tr>
            <td style="text-align:left">ICU admissions, no.</td>
            <td>{no_admissions_total}</td>
            <td>{no_admissions_ICU}</td>
            <td>{no_admissions_MCU}</td>
        </tr>    
        <tr>
            <td style="text-align:left">Surgical admissions, no. (%)</td>
            <td>{surgical_total} ({surgical_pct_total:.2f})</td>
            <td>{surgical_ICU} ({surgical_pct_ICU:.2f})</td>
            <td>{surgical_MCU} ({surgical_pct_MCU:.2f})</td>
        </tr>
        <tr>
            <td style="text-align:left">Medical admissions, no. (%)</td>
            <td>{medical_total} ({medical_pct_total:.2f})</td>
            <td>{medical_ICU} ({medical_pct_ICU:.2f})</td>
            <td>{medical_MCU} ({medical_pct_MCU:.2f})</td>
        </tr>
        <tr>
            <td style="text-align:left">Surgical/Medical Unknown, no. (%)</td>
            <td>{medical_surgical_na_total} ({medical_surgical_na_pct_total:.2f})</td>
            <td>{medical_surgical_na_ICU} ({medical_surgical_na_pct_ICU:.2f})</td>
            <td>{medical_surgical_na_MCU} ({medical_surgical_na_pct_MCU:.2f})</td>
        </tr>
        <tr>
            <td style="text-align:left">Urgent admissions, no. (%)</td>
            <td>{urgency_total} ({urgency_pct_total:.2f})</td>
            <td>{urgency_ICU} ({urgency_pct_ICU:.2f})</td>
            <td>{urgency_MCU} ({urgency_pct_MCU:.2f})</td>
        </tr>
        <tr>
            <th style="text-align:left" colspan=4>Reason for admission</td>
        </tr>
        <tr>
            <td style="text-align:left">Cardiothoracic surgery, no. (%)</td>
            <td>{reason_car_surg_total} ({reason_car_surg_pct_total:.2f})</td>
            <td>{reason_car_surg_ICU} ({reason_car_surg_pct_ICU:.2f})</td>
            <td>{reason_car_surg_MCU} ({reason_car_surg_pct_MCU:.2f})</td>
        </tr>
       <tr>
            <td style="text-align:left">Sepsis, no. (%)</td>
            <td>{reason_sepsis_total} ({reason_sepsis_pct_total:.2f})</td>
            <td>{reason_sepsis_ICU} ({reason_sepsis_pct_ICU:.2f})</td>
            <td>{reason_sepsis_MCU} ({reason_sepsis_pct_MCU:.2f})</td>
        </tr>
         <tr>
            <td style="text-align:left">Respiratory failure, no. (%)</td>
            <td>{reason_resp_failure_total} ({reason_resp_failure_pct_total:.2f})</td>
            <td>{reason_resp_failure_ICU} ({reason_resp_failure_pct_ICU:.2f})</td>
            <td>{reason_resp_failure_MCU} ({reason_resp_failure_pct_MCU:.2f})</td>
        </tr>
        <tr>
            <td style="text-align:left">Neurosurgery, no. (%)</td>
            <td>{reason_neu_surg_total} ({reason_neu_surg_pct_total:.2f})</td>
            <td>{reason_neu_surg_ICU} ({reason_neu_surg_pct_ICU:.2f})</td>
            <td>{reason_neu_surg_MCU} ({reason_neu_surg_pct_MCU:.2f})</td>
        </tr>
        <tr>
            <td style="text-align:left">Trauma, no. (%)</td>
            <td>{reason_trauma_total} ({reason_trauma_pct_total:.2f})</td>
            <td>{reason_trauma_ICU} ({reason_trauma_pct_ICU:.2f})</td>
            <td>{reason_trauma_MCU} ({reason_trauma_pct_MCU:.2f})</td>
        </tr>
        <tr>
            <td style="text-align:left">Gastro-intestinal surgery, no. (%)</td>
            <td>{reason_gi_surg_total} ({reason_gi_surg_pct_total:.2f})</td>
            <td>{reason_gi_surg_ICU} ({reason_gi_surg_pct_ICU:.2f})</td>
            <td>{reason_gi_surg_MCU} ({reason_gi_surg_pct_MCU:.2f})</td>
        </tr>
        <tr>
            <td style="text-align:left">Vascular surgery, no. (%)</td>
            <td>{reason_vasc_surg_total} ({reason_vasc_surg_pct_total:.2f})</td>
            <td>{reason_vasc_surg_ICU} ({reason_vasc_surg_pct_ICU:.2f})</td>
            <td>{reason_vasc_surg_MCU} ({reason_vasc_surg_pct_MCU:.2f})</td>
        </tr>
        <tr>
            <td style="text-align:left">Cardiac arrest, no. (%)</td>
            <td>{reason_car_arrest_total} ({reason_car_arrest_pct_total:.2f})</td>
            <td>{reason_car_arrest_ICU} ({reason_car_arrest_pct_ICU:.2f})</td>
            <td>{reason_car_arrest_MCU} ({reason_car_arrest_pct_MCU:.2f})</td>
        </tr>
        <tr>
            <td style="text-align:left">Neurologic disorders (non-traumatic), no. (%)</td>
            <td>{reason_neuro_total} ({reason_neuro_pct_total:.2f})</td>
            <td>{reason_neuro_ICU} ({reason_neuro_pct_ICU:.2f})</td>
            <td>{reason_neuro_MCU} ({reason_neuro_pct_MCU:.2f})</td>
        </tr>
        <tr>
            <td style="text-align:left">Cardiac disorders (incl. cardiogenic shock) no. (%)</td>
            <td>{reason_cardio_total} ({reason_cardio_pct_total:.2f})</td>
            <td>{reason_cardio_ICU} ({reason_cardio_pct_ICU:.2f})</td>
            <td>{reason_cardio_MCU} ({reason_cardio_pct_MCU:.2f})</td>
        </tr>
    </tbody>
</table>
""".format(
            #No. patients
            no_patients_total = no_patients_total,
            no_patients_ICU = no_patients_ICU,
            no_patients_MCU = no_patients_MCU,

            #No. admissions
            no_admissions_total = no_admissions_total,
            no_admissions_ICU = no_admissions_ICU,
            no_admissions_MCU = no_admissions_MCU,
    
            #surgical
            surgical_total = surgical_total,
            surgical_pct_total = surgical_pct_total,

            surgical_ICU = surgical_ICU,
            surgical_pct_ICU = surgical_pct_ICU,

            surgical_MCU = surgical_MCU,
            surgical_pct_MCU = surgical_pct_MCU,


            #medical
            medical_total = medical_total,
            medical_pct_total = medical_pct_total,

            medical_ICU = medical_ICU,
            medical_pct_ICU = medical_pct_ICU,

            medical_MCU = medical_MCU,
            medical_pct_MCU = medical_pct_MCU,

            #surgical/medical unknown
            medical_surgical_na_total = medical_surgical_na_total,
            medical_surgical_na_pct_total = medical_surgical_na_pct_total,

            medical_surgical_na_ICU = medical_surgical_na_ICU,
            medical_surgical_na_pct_ICU = medical_surgical_na_pct_ICU,

            medical_surgical_na_MCU = medical_surgical_na_MCU,
            medical_surgical_na_pct_MCU = medical_surgical_na_pct_MCU,

            #urgency
            urgency_total = urgency_total,
            urgency_pct_total = urgency_pct_total,

            urgency_ICU = urgency_ICU,
            urgency_pct_ICU = urgency_pct_ICU,

            urgency_MCU = urgency_MCU,
            urgency_pct_MCU = urgency_pct_MCU,

            #reason for admission

            ## cardiac surgery
            reason_car_surg_total = reason_car_surg_total,
            reason_car_surg_pct_total = reason_car_surg_pct_total,
            reason_car_surg_ICU = reason_car_surg_ICU,
            reason_car_surg_pct_ICU = reason_car_surg_pct_ICU,
            reason_car_surg_MCU = reason_car_surg_MCU,
            reason_car_surg_pct_MCU = reason_car_surg_pct_MCU,

            ## neurosurgery
            reason_neu_surg_total = reason_neu_surg_total,
            reason_neu_surg_pct_total = reason_neu_surg_pct_total,
            reason_neu_surg_ICU = reason_neu_surg_ICU,
            reason_neu_surg_pct_ICU = reason_neu_surg_pct_ICU,
            reason_neu_surg_MCU = reason_neu_surg_MCU,
            reason_neu_surg_pct_MCU = reason_neu_surg_pct_MCU,

            ## gastro-intestinal surgery
            reason_gi_surg_total = reason_gi_surg_total,
            reason_gi_surg_pct_total = reason_gi_surg_pct_total,
            reason_gi_surg_ICU = reason_gi_surg_ICU,
            reason_gi_surg_pct_ICU = reason_gi_surg_pct_ICU,
            reason_gi_surg_MCU = reason_gi_surg_MCU,
            reason_gi_surg_pct_MCU = reason_gi_surg_pct_MCU,

            ## cardiac arrest
            reason_car_arrest_total = reason_car_arrest_total,
            reason_car_arrest_pct_total = reason_car_arrest_pct_total,
            reason_car_arrest_ICU = reason_car_arrest_ICU,
            reason_car_arrest_pct_ICU = reason_car_arrest_pct_ICU,
            reason_car_arrest_MCU = reason_car_arrest_MCU,
            reason_car_arrest_pct_MCU = reason_car_arrest_pct_MCU,

            ## vascular surgery
            reason_vasc_surg_total = reason_vasc_surg_total,
            reason_vasc_surg_pct_total = reason_vasc_surg_pct_total,
            reason_vasc_surg_ICU = reason_vasc_surg_ICU,
            reason_vasc_surg_pct_ICU = reason_vasc_surg_pct_ICU,
            reason_vasc_surg_MCU = reason_vasc_surg_MCU,
            reason_vasc_surg_pct_MCU = reason_vasc_surg_pct_MCU,

            ## trauma
            reason_trauma_total = reason_trauma_total,
            reason_trauma_pct_total = reason_trauma_pct_total,
            reason_trauma_ICU = reason_trauma_ICU,
            reason_trauma_pct_ICU = reason_trauma_pct_ICU,
            reason_trauma_MCU = reason_trauma_MCU,
            reason_trauma_pct_MCU = reason_trauma_pct_MCU,

            ## sepsis
            reason_sepsis_total = reason_sepsis_total,
            reason_sepsis_pct_total = reason_sepsis_pct_total,
            reason_sepsis_ICU = reason_sepsis_ICU,
            reason_sepsis_pct_ICU = reason_sepsis_pct_ICU,
            reason_sepsis_MCU = reason_sepsis_MCU,
            reason_sepsis_pct_MCU = reason_sepsis_pct_MCU,

            ## respiratory failure
            reason_resp_failure_total = reason_resp_failure_total,
            reason_resp_failure_pct_total = reason_resp_failure_pct_total,
            reason_resp_failure_ICU = reason_resp_failure_ICU,
            reason_resp_failure_pct_ICU = reason_resp_failure_pct_ICU,
            reason_resp_failure_MCU = reason_resp_failure_MCU,
            reason_resp_failure_pct_MCU = reason_resp_failure_pct_MCU,
    
            ## neurologic disorders
            reason_neuro_total = reason_neuro_total,
            reason_neuro_pct_total = reason_neuro_pct_total,
            reason_neuro_ICU = reason_neuro_ICU,
            reason_neuro_pct_ICU = reason_neuro_pct_ICU,
            reason_neuro_MCU = reason_neuro_MCU,
            reason_neuro_pct_MCU = reason_neuro_pct_MCU,
    
            ## cardiologic disorders
            reason_cardio_total = reason_cardio_total,
            reason_cardio_pct_total = reason_cardio_pct_total,
            reason_cardio_ICU = reason_cardio_ICU,
            reason_cardio_pct_ICU = reason_cardio_pct_ICU,
            reason_cardio_MCU = reason_cardio_MCU,
            reason_cardio_pct_MCU = reason_cardio_pct_MCU
        )
    )
)


<table>
    <thead>
        <tr>
            <th style="text-align:left;">Patient Characteristics</th>
            <th style="text-align:right;">Total</th>
            <th style="text-align:right;">ICU</th>
            <th style="text-align:right;">MCU</th>
        </tr>
        <tr>
    </thead>
    <tbody>
        <tr>
            <td style="text-align:left">Distinct patients, no.</td>
            <td>20109</td>
            <td>16518</td>
            <td>4295</td>
        </tr>    
        <tr>
            <td style="text-align:left">ICU admissions, no.</td>
            <td>23106</td>
            <td>18386</td>
            <td>4720</td>
        </tr>    
        <tr>
            <td style="text-align:left">Surgical admissions, no. (%)</td>
            <td>11294 (48.88)</td>
            <td>8943 (48.64)</td>
            <td>2351 (49.81)</td>
        </tr>
        <tr>
            <td style="text-align:left">Medical admissions, no. (%)</td>
            <td>5818 (25.18)</td>
            <td>4775 (25.97)</td>
            <td>1043 (22.10)</td>
        </tr>
        <tr>
            <td style="text-align:left">Surgical/Medical Unknown, no. (%)</td>
            <td>5994 (25.94)</td>
            <td>4668 (25.39)</td>
            <td>1326 (28.09)</td>
        </tr>
        <tr>
            <td style="text-align:left">Urgent admissions, no. (%)</td>
            <td>6246 (27.03)</td>
            <td>4985 (27.11)</td>
            <td>1261 (26.72)</td>
        </tr>
        <tr>
            <th style="text-align:left" colspan=4>Reason for admission</td>
        </tr>
        <tr>
            <td style="text-align:left">Cardiothoracic surgery, no. (%)</td>
            <td>5939 (25.70)</td>
            <td>5763 (31.34)</td>
            <td>176 (3.73)</td>
        </tr>
       <tr>
            <td style="text-align:left">Sepsis, no. (%)</td>
            <td>3152 (13.64)</td>
            <td>2768 (15.05)</td>
            <td>384 (8.14)</td>
        </tr>
         <tr>
            <td style="text-align:left">Respiratory failure, no. (%)</td>
            <td>1568 (6.79)</td>
            <td>1403 (7.63)</td>
            <td>165 (3.50)</td>
        </tr>
        <tr>
            <td style="text-align:left">Neurosurgery, no. (%)</td>
            <td>1617 (7.00)</td>
            <td>737 (4.01)</td>
            <td>880 (18.64)</td>
        </tr>
        <tr>
            <td style="text-align:left">Trauma, no. (%)</td>
            <td>1241 (5.37)</td>
            <td>896 (4.87)</td>
            <td>345 (7.31)</td>
        </tr>
        <tr>
            <td style="text-align:left">Gastro-intestinal surgery, no. (%)</td>
            <td>1149 (4.97)</td>
            <td>800 (4.35)</td>
            <td>349 (7.39)</td>
        </tr>
        <tr>
            <td style="text-align:left">Vascular surgery, no. (%)</td>
            <td>1039 (4.50)</td>
            <td>793 (4.31)</td>
            <td>246 (5.21)</td>
        </tr>
        <tr>
            <td style="text-align:left">Cardiac arrest, no. (%)</td>
            <td>961 (4.16)</td>
            <td>960 (5.22)</td>
            <td>1 (0.02)</td>
        </tr>
        <tr>
            <td style="text-align:left">Neurologic disorders (non-traumatic), no. (%)</td>
            <td>627 (2.71)</td>
            <td>474 (2.58)</td>
            <td>153 (3.24)</td>
        </tr>
        <tr>
            <td style="text-align:left">Cardiac disorders (incl. cardiogenic shock) no. (%)</td>
            <td>537 (2.32)</td>
            <td>484 (2.63)</td>
            <td>53 (1.12)</td>
        </tr>
    </tbody>
</table>
